In [1]:
import matplotlib.pyplot as plt
import numpy as np
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import tensorflow_datasets as tfds
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
input_dim = 138

In [ ]:
df = pd.read_csv(r"dataset_v10.csv",index_col=[0])
df

In [ ]:
tf.convert_to_tensor(df)
# apply normalization to all features
normalized_attr = df.columns.tolist()[:-1]
normalized_attr
df[normalized_attr] = df[normalized_attr].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
#df[normalized_attr] = df[normalized_attr].apply(lambda x: (x - x.mean()) / (x.std()))

In [ ]:
import random
from datetime import datetime
random.seed(datetime.now())
train_dataset = df.sample(frac=0.75, random_state=random.randint(0,100000))
test_dataset = df.drop(train_dataset.index)
train_dataset

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('Market value')
test_labels = test_features.pop('Market value')


# autoencoder model


In [2]:
from keras import regularizers
def autoencoder_model():
    model = keras.Sequential([
    layers.Input(shape=(138,)),
    layers.Dense(64, activation='relu'),
    #layers.Dense(32, activation='relu'),
    #layers.Dense(64, activation='relu',activity_regularizer=regularizers.l1(10e-5)),
    #layers.Dense(64, activation='relu'),
    layers.Dense(138, activation='relu')
    ])

    model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(0.001))
    return model


In [3]:
auto_encoder = autoencoder_model()
auto_encoder.summary()
tf.keras.utils.plot_model(auto_encoder,"autoencode_arch.png")

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                8896      
                                                                 
 dense_1 (Dense)             (None, 138)               8970      
                                                                 
Total params: 17,866
Trainable params: 17,866
Non-trainable params: 0
_________________________________________________________________
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [ ]:
history = auto_encoder.fit(
    train_features,
    train_features,
    validation_split=0.2,
    batch_size=50,
    verbose=1, epochs=1000)

In [ ]:
test_results = {}
test_results['auto_encoder'] = auto_encoder.evaluate(test_features, test_features, verbose=0)
print(test_results['auto_encoder'] )
pd.DataFrame(test_results, index=['mse']).T

In [ ]:
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [ ]:
auto_encoder.save("auto_encoder_ls.h5")